In [ ]:
__nbid__ = '0021'
__author__ = 'Robert Nikutta <robert.nikutta@noirlab.edu>'
__version__ = '20240606' # yyyymmdd
__datasets__ = ['gogreen_dr1']
__keywords__ = ['image cutout', 'SIA']

# GOGREEN DR1 at Data Lab - Simple Image Access (SIA)

*Author:* Robert Nikutta \<robert.nikutta@noirlab.edu\>

This notebook demonstrates how the GOGREEN DR1 images can be queried for at Data Lab using the SIA service, and how cutouts around positions on the sky can be retrieved.

# Table of contents
* [Disclaimer & attribution](#attribution)
* [Imports, setup, & SIA config](#imports)
* [Family picture of a cluster](#family)
  * [Query the clusters catalog for information using TAP](#query-clusters)
  * [Run a SIA query](#sia1)
  * [Download an image](#image)
  * [Plot the image](#plot-image)
* [Gallery of cluster members](#gallery)
  * [Query the photometry table for cluster member data](#query-photometry)
  * [Calling the SIA service for individual cutouts](#sia2)
  * [Plotting a gallery of thumbnails](#plot-gallery)

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.

<a class="anchor" id="imports"></a>
# Imports, setup and SIA config

In [ ]:
# 3rd party
import numpy as np
from astropy.utils.data import download_file
from astropy.io import fits
from astropy.wcs import WCS

import matplotlib
from matplotlib.ticker import NullFormatter, NullLocator
import pylab as plt
plt.rcParams.update({'font.size': 16})

from pyvo.dal import sia
# this URL tells the SIA service where to find images and image metadata
DEF_ACCESS_URL = "https://datalab.noirlab.edu/sia/gogreen_dr1"
svc = sia.SIAService(DEF_ACCESS_URL)

# Data Lab
from dl import queryClient as qc

<a class="anchor" id="family"></a>
# Family picture of a cluster

The SIA service can be used to query the database of meta-information about images served by the cutout service. Often the metadata are in varying degrees of completeness or correctness, but we can usually make sense of it.

<a class="anchor" id="query-clusters"></a>
## Query the clusters catalog for information using TAP 

Let's first query the TAP service (Table Access Protocol) for information about clusters. This can be accomplished by issuing SQL queries with the Data Lab `queryClient` (imported as `qc`):

In [ ]:
df = qc.query("select * from gogreen_dr1.clusters",fmt='pandas')
print("Shape:", df.shape)
print("Columns:", df.columns)
df.head(10)

We have requested that the query result be converted on the fly to a Pandas data frame, and we have printed the data frame shape (26 rows = clusters, 39 columns), the column names, and the first 10 rows of the table.

Let's pick one cluster... How about SXDF49 ? (it has index number 7 in the table above):

In [ ]:
cluster = df.iloc[7]

<a class="anchor" id="sia1"></a>
## Run a SIA query

Now let's issue a SIA query to fetch information about available images that contain the coordinates of SXDF49:

In [ ]:
# select FOV and the RA and Dec center coordinates for the SIA search
fov = 1 # degrees (= 12 arcmin)
ra, dec = cluster.ra_best, cluster.dec_best

# launch the SIA search
#imgTable = svc.search((ra,dec), (fov, fov), verbosity=2).to_table()
imgTable = svc.search((ra,dec), (fov, fov), verbosity=2).to_table().to_pandas()
imgTable

Looks like we found two images within 1 degree of the RA & Dec coordinates. Of course we want the row for SXDF49 (which is the first one in the two-row table above).

In [ ]:
print(cluster.cluster)  # cluster.cluster has the cluster name
row = imgTable[imgTable['access_url'].astype(str).str.contains(cluster.cluster)]
row

<a class="anchor" id="image"></a>
## Download an image

The `access_url` field contains the URL to fetch the FITS image. We can download the file, and extract the data and header information (including the WCS):

In [ ]:
url = row['access_url'][0]
print("url = ",url)
filename = download_file(url,cache=True,show_progress=False,timeout=120)
hdu = fits.open(filename)[0]
image = hdu.data
wcs = WCS(hdu.header)

<a class="anchor" id="plot-image"></a>
## Plot the image

We are ready to plot the image. Because of the high dynamic range, we'll scale the pixel values as an hyperbolic arcsine. We'll overplot the WCS as well.

In [ ]:
fig = plt.figure(figsize=(13,13))
ax = fig.add_subplot(projection=wcs, label='overlays')
ax.imshow(np.arcsinh(image),origin='lower',cmap='bone')
ax.set_title(cluster.cluster)

lon = ax.coords[0]
lat = ax.coords[1]
lon.set_major_formatter('d.ddd')
lat.set_major_formatter('d.ddd')

ax.grid(color='white', ls='solid')

<a class="anchor" id="gallery"></a>
# Gallery of cluster members

Using the SIA image cutout service we can request small thumbnails (cutouts) of individual galaxies from a cluster. For this example we will again use cluster SXDF49.

<a class="anchor" id="query-photometry"></a>
## Query the photometry table for cluster member data
Let's first query the photometry catalog for data about members of SXDF49.

In [ ]:
dfp = qc.query("select * from gogreen_dr1.photo where cluster='SXDF49' order by cphotid limit 35",fmt='pandas')
dfp.head()

We have asked for the first 35 members of the cluster, after sorting by `cphotid`, and have also asked for the result to be returned as a Pandas data frame.

<a class="anchor" id="sia2"></a>
## Calling the SIA service for individual cutouts
Looping over the resulting table, we can request from the SIA service small cutouts (FOV = 0.005 degrees) around each set of RA & Dec coordinates.

In [ ]:
n = dfp.shape[0]
fov = 0.005
images = []
for j in range(n):
    ra, dec = dfp.iloc[j].ra, dfp.iloc[j].dec
    print("Querying object %2d/%d,ra = %8.4f, dec = %8.4f" % (j+1,n,ra,dec))
    imgTable = svc.search((ra,dec), (fov, fov), verbosity=2).to_table()
    sel = (imgTable['prodtype'] != 'wtmap')
    row = imgTable[sel][0]
    url = row['access_url']
    filename = download_file(url,cache=True,show_progress=False,timeout=120)
    hdu = fits.open(filename)[0]
    image = hdu.data
    images.append(image)

<a class="anchor" id="plot-gallery"></a>
## Plotting a gallery of thumbnails

With the images stored in the `images` list, we can now plot a 7x7 gallery. Above each thumbnail we will print the `cphotid` of the cluster member, and will print the photometric redshift `zphot` within each thumbnail.

In [ ]:
fig = plt.figure(figsize=(15,13))
for j,img in enumerate(images):
    ax = fig.add_subplot(5,7,j+1)
    ax.imshow(np.arcsinh(img.T),origin='lower',cmap='bone')
    ax.xaxis.set_major_locator(NullLocator())
    ax.yaxis.set_major_locator(NullLocator())
    ax.xaxis.set_major_formatter(NullFormatter())
    ax.yaxis.set_major_formatter(NullFormatter())
    ax.set_title(dfp['cphotid'][j])
    ax.text(0.07,0.87,'z_phot = %g' % dfp.iloc[j]['zphot'],transform=ax.transAxes,color='w',ha='left',fontsize=12)

fig.subplots_adjust(wspace=0.02,hspace=0.1)

In every thumbnail the cluster member with `cphotid` is the object at the center of the image.